In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [ ]:
aqar = pd.read_csv('RiyadhVillasAqar.csv')
aqar

# Investigation and 1st cleaning  

In [ ]:
aqar.drop(columns = 'Unnamed: 0', inplace=True)
aqar

In [ ]:
aqar.describe()

## data seems fine other than 'square price' having min with 3.4 riyals!

In [ ]:
aqar.describe(include='object')

## there are 295 neighbourhood included in this dataset, i would change the type of 'bathrooms' to be numric categories 

In [ ]:
aqar.info()

In [ ]:
aqar['neighbourhood'].value_counts()[:10]

In [ ]:
replace = lambda x: ' '.join(' '.join([c if (len(c)>1 and c !=' ' ) else '' for c in str(x).split()]).split())
aqar['neighbourhood']= aqar['neighbourhood'].apply(replace)

rename = lambda x: x if len(x) > 2 else 'unknown' 
aqar['neighbourhood']= aqar['neighbourhood'].apply(rename)
aqar['neighbourhood'].unique()

In [ ]:
len(aqar['neighbourhood'].unique())

In [ ]:
hood = {'طويق': 'Tuwik', 'الرمال': 'Ar Rimal', 'عكاظ': 'Okaz', 'النرجس': 'An Narjis',
        'بدر': 'Bader', 'العارض': 'Al Aarid', 'المونسية': 'Almunsiyah', 'الملقا': 'Al Malqa', 
        'المهدية': 'Al Mahdiyah', 'الحزم': 'Al Hazm'}

aqar['neighbourhood'].replace(hood, inplace =True)

In [ ]:
aqar['neighbourhood'].value_counts()[:10]

In [ ]:
aqar

In [ ]:
aqar['bathrooms'].replace(r'\D','', regex=True , inplace =True)
aqar['bathrooms'].value_counts()

In [ ]:
front={'شمالية': 'N','شرقية': 'E','جنوبية': 'S','غربية': 'W',
       'شمالية شرقية': 'NE','جنوبية غربية': 'SW','جنوبية شرقية': 'SE',
       'شمالية غربية': 'NE', 'ثلاث شوارع': '3S','أربع شوارع': '4S',}
aqar['front'].replace(front,inplace=True)
aqar['front'].value_counts()

In [ ]:
loc_name = {'غرب الرياض': 'W', 'جنوب الرياض': 'S', 'شرق الرياض': 'E', 'شمال الرياض': 'N'}
aqar['location'].replace(loc_name, inplace =True)
aqar['location'].value_counts()

In [ ]:
aqar['lounges'].value_counts()

In [ ]:
string = ['+', '0', '.']
for i in string:
    aqar['lounges'] = aqar['lounges'].str.replace(i,'')

aqar['lounges'].value_counts()

In [ ]:
print(len(aqar['apartments'].value_counts().index.unique()))
aqar['apartments'].value_counts().index.unique() 

In [ ]:
aqar['apartments'] = aqar['apartments'].astype("string")
print(len(aqar['apartments'].value_counts().index.unique()))
aqar['apartments'].value_counts().index.unique()

In [ ]:
for i in string:
    aqar['apartments'] = aqar['apartments'].str.replace('+','')

print(aqar['apartments'].value_counts().count())
aqar['apartments'].value_counts()

In [ ]:
def apartments(x):
    if '.' in x and x != '0.0':
        x = x.replace('0','')
        x = x.replace('.','')
        return x
    elif x == '0.0':
        return '0'
    else:
        return x
    
    
aqar['apartments'] = aqar['apartments'].apply(apartments)
print(aqar['apartments'].value_counts().count())
print(aqar['apartments'].value_counts().index.unique())

# Outliers 

In [ ]:
sns.pairplot(aqar[['price', 'square price']])

In [ ]:
sns.boxplot(aqar[['price', 'square price']])

In [ ]:
def limits(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1

    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    print("Upper Bound:",upper)   
    print("Lower Bound:",lower)
    return upper, lower
    

print("price")
priceupper, pricelower = limits(aqar['price'])
print("--------------------")
print("square price")
squareupper, squarelower = limits(aqar['square price'])

In [ ]:
#droping the outliers the 
outliers = aqar[(aqar['price'] < pricelower) | (aqar['price'] > priceupper)]
aqar = aqar.drop(outliers.index)

outliers = aqar[(aqar['square price'] < squarelower) | (aqar['square price'] > squareupper)]
aqar = aqar.drop(outliers.index)

In [ ]:
sns.pairplot(aqar[['price', 'square price']])

In [ ]:
sns.boxplot(aqar[['price', 'square price']])

# Feature Engineering 

In [ ]:
aqar['Area m2'] = aqar['price'] / aqar['square price']
aqar

In [ ]:
aqar.describe()

In [ ]:
aqar

In [ ]:
np.linspace(50, 4500, 4)

In [ ]:
def AreaGroup(x):
    if x >= 50 and x <= 400:
        return 'Normal'
    elif x > 400 and x <= 1533:
        return 'Large'
    elif x > 1533 and x <= 3016:
        return 'Huge'
    elif x > 3016:
        return 'Vast'
    
aqar['Area Rank'] = aqar['Area m2'].apply(AreaGroup)
aqar

# Missing data handeling 

In [ ]:
aqar.isnull().sum()/len(aqar) *100

In [ ]:
aqar.info()

In [ ]:
aqar['lounges'] = aqar['lounges'].astype('Int64')
meanlist = aqar.groupby('Area Rank')['lounges'].mean()
meanlist

In [ ]:
def fun(x, y):
    if pd.isnull(y):
        for i in meanlist.index:
            if i == x:
                return meanlist[x]
    else: 
        return y

aqar['lounges'] = list(map(fun, aqar['Area Rank'], aqar['lounges']))

aqar.isnull().sum()/len(aqar) *100

In [ ]:
aqar

In [ ]:
aqar['lounges'] = aqar['lounges'].astype('int64')
aqar

In [ ]:
meanlist = aqar.groupby('neighbourhood')['streetWidth'].mean()
meanlist

In [ ]:
aqar['streetWidth'] = list(map(fun, aqar['neighbourhood'], aqar['streetWidth']))
aqar.isnull().sum()/len(aqar) *100

In [ ]:
meanlist = aqar.groupby('neighbourhood')['square price'].mean()
aqar['square price'] = list(map(fun, aqar['neighbourhood'], aqar['square price']))
aqar.isnull().sum()/len(aqar) *100

In [ ]:
meanlist = aqar.groupby('neighbourhood')['price'].mean()
aqar['price'] = list(map(fun, aqar['neighbourhood'], aqar['price']))
aqar.isnull().sum()/len(aqar) *100

In [ ]:
aqar['Area m2'] = aqar['price'] / aqar['square price']
aqar.isnull().sum()/len(aqar) *100

In [ ]:
def AreaGroup(x):
    if x >= 50 and x <= 400:
        return 'Normal'
    elif x > 400 and x <= 1533:
        return 'Large'
    elif x > 1533 and x <= 3016:
        return 'Huge'
    elif x > 3016:
        return 'Vast'
    
aqar['Area Rank'] = aqar['Area m2'].apply(AreaGroup)
aqar.isnull().sum()/len(aqar) *100

In [ ]:
aqar['streetWidth'].fillna(0, inplace=True)
aqar.isnull().sum()/len(aqar) *100

# data visulization

In [ ]:
myData = aqar[(aqar['neighbourhood'] == 'Tuwik') | (aqar['neighbourhood'] == 'Ar Rimal') |
              (aqar['neighbourhood'] == 'Okaz') | (aqar['neighbourhood'] == 'An Narjis') |
              (aqar['neighbourhood'] == 'Bader') | (aqar['neighbourhood'] == 'Al Aarid') |
              (aqar['neighbourhood'] == 'Almunsiyah') | (aqar['neighbourhood'] == 'Al Malqa') |
              (aqar['neighbourhood'] == 'Al Mahdiyah') | (aqar['neighbourhood'] == 'Al Hazm')]
myData

In [ ]:
hoodSpace= myData.groupby('neighbourhood')['space'].mean()
hoodSpace = hoodSpace.sort_values(ascending=False)
hoodSpaceX = hoodSpace.index
hoodSpaceY = hoodSpace.values

hoodSpaceplot = px.bar(x=hoodSpaceX, y= hoodSpaceY,
                       color=hoodSpaceX,
                       opacity=0.7,
                       color_discrete_sequence= px.colors.sequential.Blugrn,
                       labels=dict(x='Neighbourhood', y ="Space Avg"))
hoodSpaceplot

In [ ]:
import geojson
with open("Riyadh Saudi Arabia.geojson") as f:
    Riyadh = geojson.load(f)

In [ ]:
mymap = px.choropleth_mapbox(myData, geojson=Riyadh, color=myData['neighbourhood'], opacity =0.7,
                           locations=myData['neighbourhood'], featureidkey="properties.NH",
                           center={"lat": 24.6983392, "lon": 46.6189186},
                           color_discrete_sequence = px.colors.sequential.Blugrn,
                           mapbox_style="carto-positron", zoom=8.7)
mymap.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
mymap.show()

In [ ]:
str(myData.groupby('neighbourhood')['price'].max()[0]).replace('0000.0','k')

In [ ]:
neighbourhoodAndPrice = px.pie(myData, names='neighbourhood' , values='price', hole=.3, color_discrete_sequence= px.colors.sequential.Blugrn)
neighbourhoodAndPrice.update_traces(textposition='inside', textinfo='percent+label')
neighbourhoodAndPrice

In [ ]:
scatterSpace = px.scatter(myData, x='space' , y='price', color='neighbourhood', color_discrete_sequence= px.colors.sequential.Blugrn)
scatterSpace

In [ ]:
neighbourhoodRooms = px.box(myData, x='neighbourhood', y='rooms', color= 'neighbourhood', color_discrete_sequence =px.colors.sequential.Blugrn)
neighbourhoodRooms

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = ''
for i in myData['neighbourhood'].unique():
    text = text + " "+i

cloud = WordCloud(background_color='white').generate(text)
cloud.to_file('words.png')

In [ ]:
myData

In [ ]:
NH = 'Okaz'
df = myData[myData['neighbourhood']== NH]
indexs = df['Area Rank'].value_counts().index
values = df['Area Rank'].value_counts().values

areaRank = px.area(df, x=indexs ,  y= values,color =indexs, color_discrete_sequence = px.colors.sequential.Blugrn,)
areaRank

In [ ]:
NH = 'Okaz'
df = myData[myData['neighbourhood']== NH]
indexs = df.groupby('Area Rank')['price'].mean().index
values = df.groupby('Area Rank')['price'].mean().values

bub = px.scatter(x=indexs, y=values,
        size=values, color=indexs, size_max=60,color_discrete_sequence= px.colors.sequential.Blugrn)
bub.show()

In [ ]:
NH = 'Okaz'
df = myData[myData['neighbourhood']== NH]
data = df['rooms'].value_counts()
indexs = data.index
values = data.values

area = px.area(x=indexs, y= values,
                       color=indexs,
                       color_discrete_sequence = px.colors.sequential.Blugrn,
                       labels=dict(x='Number of Rooms', y ="Count"))
area

In [ ]:
NH = 'Okaz'
df = myData[myData['neighbourhood']== NH]

histogram = px.histogram(df, x='Area Rank', y='streetWidth', color= 'Area Rank', color_discrete_sequence =px.colors.sequential.Blugrn)
histogram

In [ ]:
NH = 'Okaz'
df = myData[myData['neighbourhood']== NH]

fig = px.sunburst(df, path=['Area Rank', 'lounges'], values='lounges', color= 'lounges', color_continuous_scale =px.colors.sequential.Blugrn )
fig.show()

# Plotly Dash 

In [ ]:
from dash import html
from jupyter_dash import JupyterDash
import dash
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import base64
import dash_html_components as html

In [ ]:
def encode_image(image_file):
    test_base64 = base64.b64encode(open(image_file, 'rb').read()).decode('ascii')
    return 'data:image/png;base64,{}'.format(test_base64)

In [ ]:
def setLocation(NH):
    
    df = myData[myData['neighbourhood']== NH]
    indexs = df['Area Rank'].value_counts().index
    values = df['Area Rank'].value_counts().values
    areaRank = px.area(df, x=indexs ,  y= values,color =indexs, 
                       labels=dict(x='Area Rank', y ="Count"),
                       color_discrete_sequence = px.colors.sequential.Blugrn,)

    indexs = df.groupby('Area Rank')['price'].mean().index
    values = df.groupby('Area Rank')['price'].mean().values
    bub = px.scatter(x=indexs, y=values,
            size=values, color=indexs, size_max=60,
                     labels=dict(x='Area Rank', y ="Price"),
                     color_discrete_sequence= px.colors.sequential.Blugrn)


    data = df['rooms'].value_counts()
    indexs = data.index
    values = data.values
    area = px.area(x=indexs, y= values,
                           color=indexs,
                           color_discrete_sequence = px.colors.sequential.Blugrn,
                           labels=dict(x='Number of Rooms', y ="Count"))

    histogram = px.histogram(df, x='Area Rank', y='streetWidth', color= 'Area Rank',
                             labels=dict(x='Number of Rooms', y ="Count"),
                             color_discrete_sequence =px.colors.sequential.Blugrn)

    fig = px.sunburst(df, path=['Area Rank', 'lounges'], values='lounges', color= 'lounges', 
                      color_continuous_scale =px.colors.sequential.Blugrn )
    return areaRank, bub, area, histogram, fig

In [ ]:
app = JupyterDash(
    __name__,
    external_stylesheets = [dbc.themes.LUX, dbc.themes.SPACELAB, dbc.icons.BOOTSTRAP]
)
app.layout = html.Div([
    html.H1('Riyash villas Aqar Analysis',
           style={"text-align": "center", 'width': '100%', 'padding': '20px 5px', 'display': 'inline-block', 'font-family': "Times New Roman"}),
    dcc.Tabs(id="tabs-example-graph", value='tab-1-example-graph', children=[
        dcc.Tab(label='DataSet Dashboard', value='tab-1-example-graph'),
        dcc.Tab(label='Neighbourhood Dashboard', value='tab-2-example-graph'),
    ]),
    html.Div(id='tabs-content-example-graph')
])

@callback(Output('tabs-content-example-graph', 'children'),
              Input('tabs-example-graph', 'value'))
def render_content(tab):
    if tab == 'tab-1-example-graph':
        return html.Div([
            ####start of tab 1
            
            html.Div([
                
                ################################card group
                dbc.Row([
                    
                    dbc.Col(dbc.Card([
                                dbc.Row([

                                        dbc.Col(
                                            dbc.CardImg(
                                                src= encode_image("imgs/HP.png"),
                                                className="img-fluid rounded-start",
                                            ), className="col-md-4"), ##end of col 1

                                        dbc.Col(
                                            dbc.CardBody([
                                                html.H4(str(myData.groupby('neighbourhood')['price'].max()[0]).replace('0000.0','k'), 
                                                        className="card-title"),
                                                html.P("Higest villa Price", className="card-text"),  
                                            ]),className="col-md-8"), ##end of col 2

                                    ],className="g-0 d-flex align-items-center"),##end of row

                        ],className="text-center", color="success", outline=True, style={'height':'15vh'})
                           , width=2),
                    
                    dbc.Col(dbc.Card([
                                dbc.Row([

                                        dbc.Col(
                                            dbc.CardImg(
                                                src= encode_image("imgs/LP.png"),
                                                className="img-fluid rounded-start",
                                            ), className="col-md-4"), ##end of col 1

                                        dbc.Col(
                                            dbc.CardBody([
                                                html.H4(str(myData.groupby('neighbourhood')['price'].min()[0]).replace('0000.0','k'), className="card-title"),
                                                html.P("Lowest villa Price", className="card-text"),  
                                            ]),className="col-md-8"), ##end of col 2

                                    ],className="g-0 d-flex align-items-center")##end of row

                        ],className="text-center", color="dark", outline=True, style={'height':'15vh'})
                           , width=2),
                    
                    dbc.Col(dbc.Card([
                                dbc.Row([

                                        dbc.Col(
                                            dbc.CardImg(
                                                src= encode_image("imgs/map.png"),
                                                className="img-fluid rounded-start",
                                            ), className="col-md-4"), ##end of col 1

                                        dbc.Col(
                                            dbc.CardBody([
                                                html.H4(len(myData['neighbourhood'].value_counts()), className="card-title"),
                                                html.P("Districts", className="card-text"),  
                                            ]),className="col-md-8"), ##end of col 2

                                    ],className="g-0 d-flex align-items-center"),##end of row

                        ],className="text-center", color="success", outline=True, style={'height':'15vh'})
                           , width=2),
                    
                    dbc.Col(dbc.Card([
                                dbc.Row([

                                        dbc.Col(
                                            dbc.CardImg(
                                                src= encode_image("imgs/home.png"),
                                                className="img-fluid rounded-start",
                                            ), className="col-md-4"), ##end of col 1

                                        dbc.Col(
                                            dbc.CardBody([
                                                html.H4(len(myData), className="card-title"),
                                                html.P("Villas", className="card-text"),  
                                            ]),className="col-md-8"), ##end of col 2

                                    ],className="g-0 d-flex align-items-center"),##end of row

                        ],className="text-center", color="dark", outline=True, style={'height':'15vh'})##end of card
                        , width=2),
                    
                    
                    dbc.Col(dbc.Card([
                                    dbc.CardImg(
                                                src= encode_image("words.png"),
                                                className="img-fluid rounded-start",
                                            )
                        ],className="text-center", color="dark", outline=True, style={'height':'15vh'})##end of card
                        , width =2),
                    
                    
                ],justify="center", style={'padding': '20px'}),
                ##################################################end of card group row
                

                
                ################################################################################row 1
                dbc.Row([
                    
                    dbc.Col(dbc.Card([
                     dbc.CardBody([
                             html.H3("Location of Neighbourhood", className="text-primary", style = {'text-align':'center' }),
                             dcc.Graph(id='map', figure=mymap, style={"width": "100%", "height": "100%"})
                         ]),
                        ],style={'box-shadow': '2px 2px 2px lightgrey'})),
                    
                    dbc.Col(
                        dbc.Card([
                     dbc.CardBody([
                             html.H3('Neighbourhoods Villas Prices', className="text-primary", style = {'text-align':'center' }),
                             dcc.Graph(id='G2', figure=neighbourhoodAndPrice, style={"width": "100%", "height": "100%"})
                         ]),
                        ],style={'box-shadow': '2px 2px 2px lightgrey'})),
                    
                ], style={'padding': '10px'}),######################################################################3#########end of row 1 
                
                ########################################################################################row2 
                dbc.Row([
                    
                    dbc.Col(
                        dbc.Card([
                     dbc.CardBody([
                             html.H3('Neighbourhoods Villas Area', className="text-primary", style = {'text-align':'center' }),
                             dcc.Graph(id='G1', figure=hoodSpaceplot, style={"width": "100%", "height": "100%"})
                         ]),
                        ],style={'box-shadow': '2px 2px 2px lightgrey'})),
                    
                    dbc.Col(
                        dbc.Card([
                     dbc.CardBody([
                             html.H3('Villas Prices and Space Distribution', className="text-primary", style = {'text-align':'center' }),
                             dcc.Graph(id='G2', figure=scatterSpace, style={"width": "100%", "height": "100%"})
                         ]),
                 ],style={'box-shadow': '2px 2px 2px lightgrey'})),
                
               ],style={'padding': '10px'}),##########################################################################################end of row 2 
                
                dbc.Card([
                     dbc.CardBody([
                             html.H3('Neighbourhoods Rooms Distribution', className="text-primary", style = {'text-align':'center' }),
                             dcc.Graph(id='G1', figure=neighbourhoodRooms, style={"width": "100%", "height": "100%"})
                         ]),
                        ],style={'box-shadow': '2px 2px 2px lightgrey'}),
                
                       
      ])####end of tab row     
    ],style={'background-color': '#F2F2F2'})####end of tab 1
    
    
    elif tab == 'tab-2-example-graph':
        return html.Div([
            
            dbc.Row([
                dbc.Card([
                     dbc.CardBody([
                             html.H3('Chose a Neighbourhood to Display its Dashboared', className="text-primary", style = {'text-align':'center' }),
                             dcc.Dropdown(['Tuwik','Ar Rimal', 'Okaz','An Narjis', 'Bader',
                         'Al Aarid', 'Almunsiyah', 'Al Malqa', 'Al Mahdiyah', 'Al Hazm'], 'Tuwik', id='dropdown'),
                         ]),
                        ],style={'box-shadow': '2px 2px 2px lightgrey'})
               ],style={'padding': '10px'}), 
            
            dbc.Row([
                    
                    dbc.Col(
                        dbc.Card([
                     dbc.CardBody([
                            html.H3('No. of lounges based on Area Rank', className="text-primary", style = {'text-align':'center' }),
                            dcc.Graph(id='lastchart', figure=fig, style={"width": "100%", "height": "100%"}),
                         ]),
                        ],style={'box-shadow': '2px 2px 2px lightgrey'})),
                    
                    dbc.Col(
                        dbc.Card([
                     dbc.CardBody([
                            html.H3('The Neighbourhoods Area Rank', className="text-primary", style = {'text-align':'center' }),
                            dcc.Graph(id='AreaRank', figure=areaRank, style={"width": "100%", "height": "100%"}),
                         ]),
                 ],style={'box-shadow': '2px 2px 2px lightgrey'})),
                
               ],style={'padding': '10px'}),
            
                dbc.Row([
                    
                    dbc.Col(
                        dbc.Card([
                     dbc.CardBody([
                            html.H3('Area Rank based on Price', className="text-primary", style = {'text-align':'center' }),
                            dcc.Graph(id='buble', figure=bub, style={"width": "100%", "height": "100%"}),
                         ]),
                        ],style={'box-shadow': '2px 2px 2px lightgrey'})),
                    
                    dbc.Col(
                        dbc.Card([
                     dbc.CardBody([
                            html.H3('Street Width based on Area Rank', className="text-primary", style = {'text-align':'center' }),
                            dcc.Graph(id='histogramchart', figure=histogram, style={"width": "100%", "height": "100%"}),
                         ]),
                 ],style={'box-shadow': '2px 2px 2px lightgrey'})),
                
               ],style={'padding': '10px'}),##########################################################################################end of row 2 
                
                dbc.Card([
                     dbc.CardBody([
                            html.H3('Rooms count', className="text-primary", style = {'text-align':'center' }),
                            dcc.Graph(id='areachart', figure=area, style={"width": "100%", "height": "100%"}),
                         ]),
                        ],style={'box-shadow': '2px 2px 2px lightgrey'}),

            
        ],style={'background-color': '#F2F2F2'})####end of tab 2
    
@callback(
    [Output('AreaRank', 'figure'),
     Output('buble', 'figure'),
     Output('areachart', 'figure'),
     Output('histogramchart', 'figure'),
     Output('lastchart', 'figure')],
    Input('dropdown', 'value')
)

def update_graph(choice):
    if choice == 'Tuwik':
        areaRank, bub, area, histogram, fig = setLocation('Tuwik')
    elif choice == 'Ar Rimal':
        areaRank, bub, area, histogram, fig = setLocation('Ar Rimal')
    elif choice == 'Okaz':
        areaRank, bub, area, histogram, fig = setLocation('Okaz')
    elif choice == 'An Narjis':
        areaRank, bub, area, histogram, fig = setLocation('An Narjis')
    elif choice == 'Bader':
        areaRank, bub, area, histogram, fig = setLocation('Bader')
    elif choice == 'Al Aarid':
        areaRank, bub, area, histogram, fig = setLocation('Al Aarid')
    elif choice == 'Al Malqa':
        areaRank, bub, area, histogram, fig = setLocation('Al Malqa')
    elif choice == 'Al Mahdiyah':
        areaRank, bub, area, histogram, fig = setLocation('Al Mahdiyah')
    elif choice == 'Al Hazm':
         areaRank, bub, area, histogram, fig = setLocation('Al Hazm')
            
    return areaRank, bub, area, histogram, fig

    
if __name__ == '__main__':
    app.run_server() 